# Load libraries

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim


# Load Data

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=20, shuffle=True)


testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=20)


In [3]:
images, labels = next(iter(trainloader))
testimages, testlabels = next(iter(testloader))

In [4]:
images = images.view(images.shape[0], -1)
testimages = testimages.view(testimages.shape[0], -1)

images.shape

torch.Size([20, 784])

# Modelling 

## Relu-SoftMax

### Relu-Softmax / Adamax

In [5]:
correct = 0
total = 0


In [6]:
# Build a feed-forward network
model1 = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.Softmax(),
                      nn.Linear(64, 10))

# Define the loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model1.parameters(), lr=0.003)


epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        
        output = model1(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
        top_p, top_class = output.topk(1, dim=1)

        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        print(f'Accuracy: {accuracy.item()*100}%')        
top_p, top_class = model1(testimages).topk(1, dim=1)

equals = top_class == testlabels.view(*top_class.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


/home/adelgado/anaconda3/lib/python3.7/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Training loss: 1.497971807916959
Accuracy: 50.0%
Training loss: 1.0415674607753753
Accuracy: 44.999998807907104%
Training loss: 0.8008145861725012
Accuracy: 55.000001192092896%
Training loss: 0.6831205493311088
Accuracy: 69.9999988079071%
Training loss: 0.5153118489931027
Accuracy: 89.99999761581421%
Accuracy: 85.00000238418579%


### Relu-Softmax / RMSProp

In [7]:
# Build a feed-forward network
model3 = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.Softmax(),
                      nn.Linear(64, 10))

# Define the loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model3.parameters(), lr=0.003)


epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
    
        # TODO: Training pass
        optimizer.zero_grad()
        
        output = model3(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
        top_p, top_class = output.topk(1, dim=1)

        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        print(f'Accuracy: {accuracy.item()*100}%')        
        
        
top_p, top_class = model3(testimages).topk(1, dim=1)

equals = top_class == testlabels.view(*top_class.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


Training loss: 2.3024120899041494
Accuracy: 10.000000149011612%
Training loss: 2.3021423166592916
Accuracy: 25.0%
Training loss: 2.3021230963071186
Accuracy: 20.000000298023224%
Training loss: 2.3023310285409293
Accuracy: 0.0%
Training loss: 2.3021843106746673
Accuracy: 30.000001192092896%
Accuracy: 15.000000596046448%


### Relu-Softmax / SGD

In [8]:
# Build a feed-forward network
model5 = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.Softmax(),
                      nn.Linear(64, 10))

# Define the loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model5.parameters(), lr=0.003)


epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
    
        # TODO: Training pass
        optimizer.zero_grad()
        
        output = model5(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
        top_p, top_class = output.topk(1, dim=1)

        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        print(f'Accuracy: {accuracy.item()*100}%')        
        
        
top_p, top_class = model5(testimages).topk(1, dim=1)

equals = top_class == testlabels.view(*top_class.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


Training loss: 2.302479820330938
Accuracy: 20.000000298023224%
Training loss: 2.300350705862045
Accuracy: 5.000000074505806%
Training loss: 2.299140646696091
Accuracy: 5.000000074505806%
Training loss: 2.296819148461024
Accuracy: 20.000000298023224%
Training loss: 2.266268878062566
Accuracy: 25.0%
Accuracy: 25.0%


## Relu-Softmax-Relu

### Relu-Soft-Relu / Adamax

In [9]:
# Build a feed-forward network
model8 = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.Softmax(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10))


# Define the loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model8.parameters(), lr=0.003)


epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
    
        # TODO: Training pass
        optimizer.zero_grad()
        
        output = model8(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
        top_p, top_class = output.topk(1, dim=1)

        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        print(f'Accuracy: {accuracy.item()*100}%')
        
        
top_p, top_class = model8(testimages).topk(1, dim=1)

equals = top_class == testlabels.view(*top_class.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


Training loss: 0.7511923342930774
Accuracy: 94.9999988079071%
Training loss: 0.2120493494841891
Accuracy: 94.9999988079071%
Training loss: 0.1489307107553662
Accuracy: 80.0000011920929%
Training loss: 0.12036036172361735
Accuracy: 100.0%
Training loss: 0.10419753631049146
Accuracy: 100.0%
Accuracy: 100.0%


In [10]:
top_p, top_class = model8(testimages).topk(1, dim=1)

equals = top_class == testlabels.view(*top_class.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


Accuracy: 100.0%


Tarda 1'49"

### Relu-Soft-Relu / RMSprop

In [11]:
# Build a feed-forward network
model10 = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.Softmax(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10))


# Define the loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model10.parameters(), lr=0.003)


epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
    
        # TODO: Training pass
        optimizer.zero_grad()
        
        output = model10(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
        top_p, top_class = output.topk(1, dim=1)

        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        print(f'Accuracy: {accuracy.item()*100}%')        
        
top_p, top_class = model10(testimages).topk(1, dim=1)

equals = top_class == testlabels.view(*top_class.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


Training loss: 2.301841409921646
Accuracy: 10.000000149011612%
Training loss: 2.301830239375432
Accuracy: 30.000001192092896%
Training loss: 2.3017399570941923
Accuracy: 10.000000149011612%
Training loss: 2.301629289865494
Accuracy: 10.000000149011612%
Training loss: 2.3017541761398315
Accuracy: 15.000000596046448%
Accuracy: 15.000000596046448%


### Relu-Soft-Relu / SGD

In [12]:
# Build a feed-forward network
model12 = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Linear(256, 100),
                      nn.ReLU(),
                      nn.Linear(100, 32),
                      nn.Softmax(),
                      nn.Linear(32, 10))


# Define the loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model12.parameters(), lr=0.003)


epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
    
        # TODO: Training pass
        optimizer.zero_grad()
        
        output = model12(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
        top_p, top_class = output.topk(1, dim=1)

        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        print(f'Accuracy: {accuracy.item()*100}%')        
        
        
top_p, top_class = model12(testimages).topk(1, dim=1)

equals = top_class == testlabels.view(*top_class.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


Training loss: 2.303156809091568
Accuracy: 20.000000298023224%
Training loss: 2.30074604956309
Accuracy: 15.000000596046448%
Training loss: 2.299807936112086
Accuracy: 15.000000596046448%
Training loss: 2.298934292236964
Accuracy: 0.0%
Training loss: 2.297786432981491
Accuracy: 5.000000074505806%
Accuracy: 15.000000596046448%


# Predicting

In [ ]:
%matplotlib inline
import helper
images, labels = next(iter(trainloader))
img = images[0].view(1, 784)
logps = model8(img)
ps = torch.exp(logps)
helper.view_classify(img.view(1, 28, 28), ps)